In [29]:
import csv
import math
import numpy as np
import cv2

In [16]:
def nanoToSecond(nano):
    return nano / 1e9

In [31]:
def nanoToMilli(nano):
    return nano / 1e6

In [19]:
def getOrientation(orientation_curr, gyro, t_prev, t_curr):
    dt = nanoToSecond(t_curr - t_prev)
    orientation = np.array(orientation_curr + gyro * dt)

    for i in range(3):
        while orientation[i] >= (2*math.pi):
            orientation[i] -= (2*math.pi)
        while orientation[i] <= -(2*math.pi):
            orientation[i] += (2*math.pi)            

    return orientation

# Gyroscope Integration
Method of computing is gyroscopic intergration then store orientation data to CSV file

In [28]:
read_file = open('visualization/data/VID_20220612_035819gyro.csv', 'r')
write_file = open('visualization/data/VID_20220612_035819_orientation.csv',  'w+', newline="")
reader = csv.reader(read_file)
writer = csv.writer(write_file)

"""Loop through the gyro data file and get orientation"""
orientation = np.array([0, 0, math.pi/2]) #in rad
counter = 0
t_prev = 0
for row in reader:
    if(counter == 0):
        counter = 1
        t_prev = int(row[3])
        continue
    orientation = getOrientation(orientation_curr=orientation, 
                                gyro=np.array([float(row[0]), float(row[1]),float(row[2])]), 
                                t_prev=t_prev, 
                                t_curr=int(row[3]))
    o = orientation * (180/math.pi)
    newEntry = np.array([o[0], o[1], o[2], row[3]])
    writer.writerow(newEntry)

read_file.close()
write_file.close()


# Process data to match video
Process new data to match video frame
[DEPRECATED]

In [36]:
def getdTcsv(reader):
    val = 0
    counter = 0
    for row in reader:
        val = int(row[3]) - val
        if counter == 1:
            break
        counter += 1
    return val
        

In [37]:
fileName = 'VID_20220612_035819'

read_file = open('visualization/data/' + fileName + '_orientation.csv', 'r')
write_file = open('visualization/data/' + fileName + '_processed.csv',  'w+', newline="")
reader = csv.reader(read_file)
writer = csv.writer(write_file)

cap = cv2.VideoCapture('visualization/data/' + fileName + '.mp4')

dt_csv = nanoToMilli(getdTcsv(reader)) #Assumed constant throughout entire CSV ()millisecond)
curr_csv_time = 0 #Unit is in millisecond

"""Loop through the gyro data file and get orientation"""
frame_exists, curr_frame = cap.read() #Get first frame

for row in reader: #Skip first row since no dt
    if not frame_exists:
        break
    
    frame_time = cap.get(cv2.CAP_PROP_POS_MSEC) #This time is in milliseconds
    if curr_csv_time > frame_time:
      #Add to new process CSV file and then go to next frame
      writer.writerow(row)
      frame_exists, curr_frame = cap.read()
    elif (curr_csv_time + dt_csv > frame_time) and (curr_csv_time + float(dt_csv/2.0) <= frame_time):
      #Add to new process CSV file and then go to next frame
      writer.writerow(row)
      frame_exists, curr_frame = cap.read()
    curr_csv_time += dt_csv

read_file.close()
write_file.close()


### Check if we have 1-on-1 matching

In [40]:
fileName = 'VID_20220612_035819'
#Check if number of rows in CSV matches number of frames so we have a 1-on-1 match
with open('visualization/data/' + fileName + '_processed.csv') as file:
    reader = csv.reader(file)
    csv_data = list(reader)

total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
assert(total_frames == len(csv_data))

# Sensor Fusion Orientation

In [1]:
import csv
import math
import numpy as np
import cv2